# BAT.jl Tutorial - Poisson Counting Experiment

In [ ]:
using BAT
using Distributions 
using IntervalSets
using ShapesOfVariables
using Plots
using ArraysOfArrays
using StatsBase 

## The Situation
We want to measure a source in the presence of background. 
For example, this could be a radioactive element that is to be measured in a laboratory and is therefore background from natural radioactivity.


## 1. Background only measurement
A measurement without the signal source yields a number of $N_B=10$ counts.
### Task: 
Perform a Bayesian analysis of this situation to estimate the parameter describing the backound count rate $\lambda_b$.

In [ ]:
struct Background<:AbstractDensity
    k::Float64 # observed counts
end

function BAT.density_logval(target::Background, params::Union{NamedTuple,AbstractVector{<:Real}})
    return logpdf(Poisson(params[1][1]), target.k) # poisson log-likelihood
end

In [ ]:
# Number of observed background events
kb = 10
likelihood_B = Background(kb)

prior_B = NamedPrior(
    λb = 0..30
)

posterior_B = PosteriorDensity(likelihood_B, prior_B)

In [ ]:
algorithm = MetropolisHastings()
nchains = 8
nsamples = 10^5

In [ ]:
samples_B, sampleids_B, stats_B, chains_B = rand(MCMCSpec(algorithm, posterior_B), nsamples, nchains);

In [ ]:
par_names=["\$\\lambda_b\$"]
#plot(posterior_B, samples_B, :λb, xlabel = par_names[1], ylabel = "P($(par_names[1]))")
#plot!(prior_B, :λb)

In [ ]:
print(stats_B)

Questions:
* What is the distribution of the posterior? What is the best estimator for the parameter?

## 2. Further Background only measurement
A second measurement without the signal source yields a number of $N_B=8$ counts.

Use posterior of first backoground measurement as prior (analytic)

In [ ]:
# replace by analytic poisson prior
hist_10 = fit(Histogram, flatview(samples_B.params)[1, :], FrequencyWeights(samples_B.weight), nbins = 400, closed = :left)

In [ ]:
# Number of observed background events
kb2 = 8
likelihood_B2 = Background(kb2)

prior_B2 = NamedPrior(
    λb = BAT.HistogramAsUvDistribution(hist_10) # replace by analytic poisson prior
)

prior_B2flat = NamedPrior(
    λb = 0..30
)

posterior_B2 = PosteriorDensity(likelihood_B2, prior_B2)
likelipost_B2 = PosteriorDensity(likelihood_B2, prior_B2flat)
;

In [ ]:
samples_B2, sampleids_B2, stats_B2, chains_B2 = rand(MCMCSpec(algorithm, posterior_B2), nsamples, nchains);
likelipost_samples_B2, likelids_B2, likeli_stats_B2, like_chains_B2 = rand(MCMCSpec(algorithm, likelipost_B2), nsamples, nchains);

In [ ]:
stats_B2.mode

In [ ]:
plot(posterior_B2, samples_B2, :λb, xlabel = par_names[1], ylabel = "P($(par_names[1]))")
#plot!(likelipost_B2, likelipost_samples_B2, :λb, seriestype=:stephist, linecolor=:blue,linewidth=1.5, localmode=false, label="likelihood")
#plot!(prior_B2, :λb, linewidth=1.5)

## 3. Signal + Background
Measurement with source: $N_{S+B}=12$

In [ ]:
struct SignalAndBackground<:AbstractDensity
    k::Float64 # observed counts
end

function BAT.density_logval(target::SignalAndBackground, params::Union{NamedTuple,AbstractVector{<:Real}})
    return logpdf(Poisson(params[1][1] + params[2][1]), target.k)  # poisson log-likelihood
end

kSB = 12
likelihood_SB = SignalAndBackground(kSB)

In [ ]:
hist_B2 = fit(Histogram, flatview(samples_B2.params)[1, :], FrequencyWeights(samples_B2.weight), nbins = 400, closed = :left)
B2 = BAT.HistogramAsUvDistribution(hist_B2);

In [ ]:
prior_SB = NamedPrior(
    λb = B2,
    λs = 0..30
)

posterior_SB = PosteriorDensity(likelihood_SB, prior_SB);

In [ ]:
samples_SB, sampleids_SB, stats_SB, chains_SB = rand(MCMCSpec(algorithm, posterior_SB), nsamples, nchains);

In [ ]:
plot(samples_SB)

## Error propagation

We want to caluclate the cross section of the signal process using the formula
# $σ = \frac{λ_s}{ε \cdot L}$
The value of $L$ is set to $1.1$.

### a) use $ϵ \propto $ Normaldistribution
The efficiency has been measured to be $ε = 0.1 \pm 0.02$.
Assume the error to follow a normal distribution and caluclate $σ$. 

In [ ]:
nsamples=800000
ε = rand(Normal(0.1,0.02),nsamples)
L = 1.1
σS = (samples_SB.params.data[2,1:nsamples])./(ε*L)
hist_σ = fit(Histogram, σS, FrequencyWeights(samples_SB.weight), nbins = 400, closed = :left)
plot(hist_σ,xlim=(0,100))

### b) Binomial analysis of calibration measurement with known source (neglect background because of high signal rate)
assume: expected calibration source rate $N_\text{expected} = 1000$ and measured by the detector $N_\text{measured} = 123$

In [ ]:
struct BinomialModel<:AbstractDensity
    n::Float64 # n trials
    k::Float64 # k succes
end

function BAT.density_logval(target::BinomialModel, params::Union{NamedTuple,AbstractVector{<:Real}})
    return logpdf(Binomial(target.n, params[1][1]), target.k) # poisson log-likelihood
end

likelihood_binomial = BinomialModel(100, 13)

In [ ]:
prior_binomial = NamedPrior(
    p = 0..1
)

posterior_binomial = PosteriorDensity(likelihood_binomial, prior_binomial);

In [ ]:
samples_binomial, sampleids_binomial, stats_binomial, chains_binomial = rand(MCMCSpec(algorithm, posterior_binomial), nsamples, nchains);

In [ ]:
plot(samples_binomial, 1)